In [2]:
import pandas as pd
import numpy as np
import helpers

In [3]:
helpers.download_data()

All files downloaded successfully


In [4]:
df = helpers.assemble_data_frame()

In [13]:
df.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,8CD5DE2C2B6C4CFC,docked_bike,2020-06-13 23:24:48,2020-06-13 23:36:55,Wilton Ave & Belmont Ave,117,Damen Ave & Clybourn Ave,163.0,41.94018,-87.65304,41.931931,-87.677856,casual


In [10]:
#Percentage of rides without returns
df['end_station_name'].isna().sum() / df.size


0.008575757148007113

In [16]:
#Percentage of members vs casual users
df['member_casual'].value_counts(normalize=True)

member    0.569854
casual    0.430146
Name: member_casual, dtype: float64

In [22]:
covid = pd.read_csv('data/covid/COVID-19_Daily_Cases__Deaths__and_Hospitalizations.csv')

In [23]:
covid

,Date,Cases - Total,Deaths - Total,Hospitalizations - Total,Cases - Age 0-17,Cases - Age 18-29,Cases - Age 30-39,Cases - Age 40-49,Cases - Age 50-59,Cases - Age 60-69,...,Hospitalizations - Age Unknown,Hospitalizations - Female,Hospitalizations - Male,Hospitalizations - Unknown Gender,Hospitalizations - Latinx,Hospitalizations - Asian Non-Latinx,Hospitalizations - Black Non-Latinx,Hospitalizations - White Non-Latinx,Hospitalizations - Other Race Non-Latinx,Hospitalizations - Unknown Race/Ethnicity
0,06/12/2021,46,5,8.0,5,13,13,3,7,1,...,0.0,3.0,5.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0
1,06/01/2022,1338,3,26.0,323,257,260,161,149,107,...,0.0,15.0,11.0,0.0,6.0,1.0,6.0,12.0,0.0,1.0
2,07/03/2022,442,0,13.0,48,98,90,63,52,42,...,0.0,6.0,7.0,0.0,3.0,1.0,3.0,5.0,0.0,1.0
3,07/06/2020,282,5,31.0,23,79,66,41,35,21,...,0.0,17.0,14.0,0.0,7.0,1.0,16.0,6.0,0.0,1.0
4,07/14/2022,464,0,NaN,73,80,85,70,55,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,06/01/2021,128,5,23.0,29,23,25,26,16,5,...,0.0,10.0,13.0,0.0,5.0,0.0,14.0,4.0,0.0,0.0
864,05/29/2020,459,20,50.0,33,114,70,85,75,42,...,0.0,31.0,19.0,0.0,12.0,1.0,26.0,9.0,1.0,1.0
865,10/10/2021,173,6,25.0,31,38,32,22,21,15,...,0.0,14.0,11.0,0.0,7.0,0.0,13.0,5.0,0.0,0.0
866,05/16/2020,514,28,102.0,35,106,87,93,77,59,...,0.0,51.0,51.0,0.0,41.0,4.0,32.0,17.0,2.0,6.0
